In [1]:
import os
import struct
from array import array

In [2]:
class MNIST(object):
    def __init__(self,path='.'):
        self.path=path
        
        self.test_img_fname='t10k-images-idx3-ubyte'
        self.test_lbl_fname='t10k-labels-idx1-ubyte'

        self.train_img_fname='train-images-idx3-ubyte'
        self.train_lbl_fname='train-labels-idx1-ubyte'

        self.test_images=[]
        self.test_labels=[]

        self.train_images=[]
        self.train_labels=[]

    def load_testing(self):
        ims,labels=self.load(os.path.join(self.path,self.test_img_fname),os.path.join(self.path,self.test_lbl_fname))
        self.test_images=ims
        self.test_labels=labels

        return ims,labels

    def load_training(self):
        ims,labels=self.load(os.path.join(self.path,self.train_img_fname),os.path.join(self.path,self.train_lbl_fname))
        self.train_images=ims
        self.train_labels=labels

        return ims,labels
    @classmethod
    def load(cls,path_img,path_lbl):
        with open(path_lbl,'rb') as file:
            magic,size=struct.unpack(">II",file.read(8))
            if magic!=2049:
                raise ValueError('Magic number mismatch, expected 2049,''got {}'.format(magic))

            labels=array("B",file.read())
            
        with open(path_img,'rb') as file:
            magic,size,rows,cols=struct.unpack(">IIII",file.read(16))
            if magic!=2051:
                raise ValueError('Magic number mismatch, expected 2051,''got {}'.format(magic))

            image_data=array("B",file.read())

        images=[]
        for i in range(size):
            images.append([0] * rows * cols)

        for i in range(size):
            images[i][:]=image_data[i * rows * cols:(i+1) * rows * cols]

        return images,labels

    @classmethod
    def display(cls,img,width=28,threshold=200):
        render=''
        for i in range(len(img)):
            if i % width ==0:
                render += '\n'
            if img[i] > threshold:
                render += '@'
            else:
                render+='.'
        return render

K Nearest Neighbors (KNN)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score 

In [4]:
mnist = MNIST()
train_images, train_labels = mnist.load_training()
test_images, test_labels = mnist.load_testing()

In [10]:
knn = KNeighborsClassifier(n_neighbors=5)

In [11]:
knn.fit(train_images, train_labels)

KNeighborsClassifier()

In [12]:
predictions = knn.predict(test_images)

In [13]:
accuracy = accuracy_score(test_labels, predictions)
print("KNN Accuracy:", accuracy) 

KNN Accuracy: 0.9688


Random Forest Classifier (RFC)

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
rfc = RandomForestClassifier(n_estimators=100)

In [16]:
rfc.fit(train_images, train_labels)

RandomForestClassifier()

In [17]:
predictions_rfc = rfc.predict(test_images)

In [18]:
accuracy_rfc = accuracy_score(test_labels, predictions_rfc)
print("Random Forest Accuracy:", accuracy_rfc)

Random Forest Accuracy: 0.9677


Support Vector Machine (SVM)

In [19]:
from sklearn import svm

In [20]:
svm_classifier = svm.SVC()

In [21]:
svm_classifier.fit(train_images, train_labels)

SVC()

In [22]:
predictions_svm = svm_classifier.predict(test_images)

In [23]:
accuracy_svm = accuracy_score(test_labels, predictions_svm)
print("SVM Accuracy:", accuracy_svm)

SVM Accuracy: 0.9792


Convolutional Neural Network (CNN)

In [24]:
import os
import struct
from array import array

In [25]:
class MNIST(object):
    def __init__(self, path='.'):
        self.path = path
        
        self.test_img_fname = 't10k-images-idx3-ubyte'
        self.test_lbl_fname = 't10k-labels-idx1-ubyte'

        self.train_img_fname = 'train-images-idx3-ubyte'
        self.train_lbl_fname = 'train-labels-idx1-ubyte'

        self.test_images = []
        self.test_labels = []

        self.train_images = []
        self.train_labels = []


    def load_testing(self):
        ims, labels = self.load(os.path.join(self.path, self.test_img_fname), os.path.join(self.path, self.test_lbl_fname))
        self.test_images = ims
        self.test_labels = labels
        return ims, labels

    def load_training(self):
        ims, labels = self.load(os.path.join(self.path, self.train_img_fname), os.path.join(self.path, self.train_lbl_fname))
        self.train_images = ims
        self.train_labels = labels
        return ims, labels
    
    @classmethod
    def load(cls, path_img, path_lbl):
        with open(path_lbl, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))

            labels = array("B", file.read())
            
        with open(path_img, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))

            image_data = array("B", file.read())

        images = []
        for i in range(size):
            images.append([0] * rows * cols)

        for i in range(size):
            images[i][:] = image_data[i * rows * cols:(i+1) * rows * cols]

        return images, labels

    @classmethod
    def display(cls, img, width=28, threshold=200):
        render = ''
        for i in range(len(img)):
            if i % width == 0:
                render += '\n'
            if img[i] > threshold:
                render += '@'
            else:
                render += '.'
        return render

In [26]:
mnist = MNIST()
train_images, train_labels = mnist.load_training()
test_images, test_labels = mnist.load_testing()

In [27]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
  # Assuming you have MNIST class defined in a separate module

class CNN_MNIST(object):
    def __init__(self):
        self.model = self.build_model()

    def build_model(self):
        model = Sequential([
            Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
            MaxPooling2D(pool_size=(2, 2)),
            Conv2D(64, kernel_size=(3, 3), activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Flatten(),
            Dense(128, activation='relu'),
            Dense(10, activation='softmax')
        ])
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    def train(self, train_images, train_labels):
        train_images = np.array(train_images).reshape(-1, 28, 28, 1) / 255.0
        train_labels = to_categorical(train_labels, num_classes=10)
        self.model.fit(train_images, train_labels, epochs=5, batch_size=64)

    def evaluate(self, test_images, test_labels):
        test_images = np.array(test_images).reshape(-1, 28, 28, 1) / 255.0
        test_labels = to_categorical(test_labels, num_classes=10)
        return self.model.evaluate(test_images, test_labels)

if __name__ == "__main__":
    # Assuming you have already loaded MNIST data using MNIST class
    mnist_data = MNIST()
    train_images, train_labels = mnist_data.load_training()
    test_images, test_labels = mnist_data.load_testing()

    cnn = CNN_MNIST()
    cnn.train(train_images, train_labels)
    test_loss, test_acc = cnn.evaluate(test_images, test_labels)
    print("Test accuracy:", test_acc)


C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 23s 22ms/step - accuracy: 0.8998 - loss: 0.3481
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - accuracy: 0.9842 - loss: 0.0522
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.9893 - loss: 0.0340
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.9927 - loss: 0.0241
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - accuracy: 0.9944 - loss: 0.0170
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9847 - loss: 0.0454
Test accuracy: 0.9887999892234802


In [28]:
print("Test accuracy:", test_acc)

Test accuracy: 0.9887999892234802
